In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!ls

'02 What is a Robot Questions.docx'
 129-yazyk-programmirovaniya-s-lekcii-i-uprazhneniya-1.rar
 129-yazyk-programmirovaniya-s-lekcii-i-uprazhneniya.rar
 2стр.docx
 2стр.docx.gdoc
 2стр.gdoc
 AbitSmarter_protected.exe
'A cassette player.docx'
 adjustable-parallax-effect-to-site
 adv.html
 airsim_rec_1.csv
 airsim_rec_big.csv
 airsim_rec.csv
 airsim_rec.txt
 app
 BCS
'BCS Плата за обучение и услуги Контракт 2016-2017.gdoc'
 Bibliography.docx
'blackbear_-_idfc_tarro_remix_(zf.fm) (1).mp3'
'blackbear_-_idfc_tarro_remix_(zf.fm).mp3'
'Camp Green Lake.docx'
 car
 car-detect
 Cars.ipynb
 cell_images
'CodeEdit Setup 0.0.5.exe'
'Colab Notebooks'
 datasets
 Debug
'Destination B2. Grammar and Vocabulary  with answer key.rar'
 destination_b2_grammar_and_vocabulary_with_keys.pdf
 dfdfdfdfd.txt
 dp
 driving_log.csv
'Easy Notes Setup 1.0.0.exe'
 epic-inventory-12-complete.zip
 epic_inventory.rar
 Ethics.mp4
 example.html
 fevzi
 Files1
 Files1.exe
 FineScanner
 frog
 Game_Data
'gg (1).jar'
 gg.jar
'He

In [9]:
cd drive/My Drive/

[Errno 2] No such file or directory: 'drive/My Drive/'
/content/drive/My Drive


In [0]:
import os
import numpy as np
import shutil

In [0]:
DATASET_PATH = 'cell_images/'

## The path to store the train images
TRAIN_PATH = './datasets/cimages_train'

## The path to store the validation images
VAL_PATH = './datasets/cimages_val'

## The path to store the test images
TEST_PATH = './datasets/cimages_test'

## The ratio of data to go into training set
TRAIN_SPLIT = 0.8

## The ratio of data to go into validation set
VAL_SPLIT = 0.1

## The number of epochs
EPOCHS = 50

## The learning rate
LR = 1e-1

## The default batch size
BS = 64

BATCH_NORM = True

In [0]:
import os
from os import listdir

def list_images_from_path(path):
    files = []

    if os.path.isfile(path):
        return files.append(path)

    if path[-1] != '/':
        path += '/'

    for file in listdir(path):
        if os.path.isfile(path + file):
            files.append(path + file)
        else:
            files += list_images_from_path(path + file)

    return files

In [0]:
def create_datasets():
    images = list_images_from_path(DATASET_PATH)

    np.random.shuffle(images)

    train_split_till = int(len(images) * TRAIN_SPLIT)
    train_images = images[:train_split_till]
    test_images = images[train_split_till:]

    val_split_till = int(len(train_images) * VAL_SPLIT)
    val_images = train_images[:val_split_till]
    train_images = train_images[val_split_till:]

    print ('# Train Images: {}/{}'.format(len(train_images), len(images)),
           '\n# Validation Images: {}/{}'.format(len(val_images), len(images)),
           '\n# Test Images: {}/{}'.format(len(test_images), len(images)))

    datasets = [
        ('train', train_images, TRAIN_PATH),
        ('val', val_images, VAL_PATH),
        ('test', test_images, TEST_PATH)
    ]

    # Loop over datasets
    for (d_type, image_paths, base_path) in datasets:
        print ('[INFO]Building {} split'.format(d_type))

        if not os.path.exists(base_path):
            print ('[INFO] Creating {} directory'.format(base_path))
            os.makedirs(base_path)

        for inp_path in image_paths:
            filename = inp_path.split('/')[-1]
            label = inp_path.split('/')[-2]

            label_path = os.path.sep.join([base_path, label])

            if not os.path.exists(label_path):
                print ('[INFO] Creating {} directory'.format(label_path))
                os.makedirs(label_path)

            path_to_new_image = os.path.sep.join([label_path, filename])
            shutil.copy2(inp_path, path_to_new_image)

    print ('[INFO]Generated Train Validation and Test Splits Successfully!!')
create_datasets()

# Train Images: 19844/27560 
# Validation Images: 2204/27560 
# Test Images: 5512/27560
[INFO]Building train split
[INFO] Creating ./datasets/cimages_train directory
[INFO] Creating ./datasets/cimages_train/Parasitized directory
[INFO] Creating ./datasets/cimages_train/Uninfected directory
[INFO]Building val split
[INFO] Creating ./datasets/cimages_val directory
[INFO] Creating ./datasets/cimages_val/Parasitized directory
[INFO] Creating ./datasets/cimages_val/Uninfected directory
[INFO]Building test split
[INFO] Creating ./datasets/cimages_test directory
[INFO] Creating ./datasets/cimages_test/Uninfected directory
[INFO] Creating ./datasets/cimages_test/Parasitized directory
[INFO]Generated Train Validation and Test Splits Successfully!!


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.initializers import glorot_normal
from keras.utils import np_utils

from keras import backend as K
def base_model():
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same', input_shape=(64, 64 , 3), name='block1_conv1'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3), padding='same', name='block1_conv2'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

    model.add(Conv2D(128, (3, 3), padding='same', name='block2_conv1'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(128, (3, 3), padding='same', name='block2_conv2'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

    model.add(Conv2D(256, (3, 3), padding='same', name='block3_conv1'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(256, (3, 3), padding='same', name='block3_conv2'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(256, (3, 3), padding='same', name='block3_conv3'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))


    model.add(Flatten())

    model.add(Dense(4096))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(4096, name='fc2'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(2))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('softmax'))

    opt = SGD(lr=0.001, momentum=0.9)

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [0]:
import argparse
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD


def lr_decay(epoch):
    epoch += 1
    power = 1.0
    alpha = LR * (1 - (epoch / float(EPOCHS))) ** power
    return alpha

if __name__ == '__main__':

    # Initialize the ImageDataGenerator
    train_aug = ImageDataGenerator(
                    rescale=1.0 / 255,
                    rotation_range=20,
                    zoom_range=0.05,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    horizontal_flip=True,
                    fill_mode="nearest")

    val_aug = ImageDataGenerator(rescale=1/255)

    # Initialize the training generator
    train_gen = train_aug.flow_from_directory(
                    TRAIN_PATH,
                    class_mode="categorical",
                    target_size=(64, 64),
                    color_mode='rgb',
                    shuffle=True,
                    batch_size=BS)

    # Intialize the validation generator
    val_gen = val_aug.flow_from_directory(
                    VAL_PATH,
                    class_mode="categorical",
                    target_size=(64, 64),
                    color_mode="rgb",
                    shuffle=False,
                    batch_size=BS)

    # Initialize the test generator
    test_gen = val_aug.flow_from_directory(
                    TEST_PATH,
                    class_mode="categorical",
                    target_size=(64, 64),
                    color_mode="rgb",
                    shuffle=False,
                    batch_size=BS)



Found 19843 images belonging to 2 classes.
Found 2203 images belonging to 2 classes.
Found 5512 images belonging to 2 classes.


In [0]:
# Training
model = ResNet.build(64, 64, 3, 2, (3, 4, 6),
(64, 128, 256, 512), reg=0.0005)
opt = SGD(lr=0.001, momentum=0.9)
model.compile(loss="binary_crossentropy", optimizer=opt,
metrics=["accuracy"])



fp = 'malaria-model-{epoch:02d}-{val_loss:.2f}.hdf5'



H = model.fit_generator(
  train_gen,
  steps_per_epoch=300,
  validation_data=val_gen,
  validation_steps=100,
  epochs=EPOCHS)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
300/300 [==============================] - 4368s 15s/step - loss: 1.1127 - acc: 0.6904 - val_loss: 0.9845 - val_acc: 0.7841
Epoch 2/50
300/300 [==============================] - 135s 451ms/step - loss: 0.7625 - acc: 0.9160 - val_loss: 0.7432 - val_acc: 0.9318
Epoch 3/50
300/300 [==============================] - 134s 447ms/step - loss: 0.6958 - acc: 0.9394 - val_loss: 0.6890 - val_acc: 0.9486
Epoch 4/50
300/300 [==============================] - 135s 450ms/step - loss: 0.6742 - acc: 0.9520 - val_loss: 0.6881 - val_acc: 0.9443
Epoch 5/50
300/300 [==============================] - 134s 447ms/step - loss: 0.6675 - acc: 0.9516 - val_loss: 0.6695 - val_acc: 0.9534
Epoch 6/50
300/300 [==============================] - 136s 452ms/step - loss: 0.6629 - acc: 0.9541 - val_loss: 0.6675 - val_acc: 0.9577
Epoch 7/50
300/300 [==============================] - 135s 449ms/

In [0]:
import keras 
model = base_model()




fp = 'malaria-model-{epoch:02d}-{val_loss:.2f}.hdf5'


callbacks = [#K.LearningRateScheduler(lr_decay, verbose=1),
             keras.callbacks.ModelCheckpoint(filepath=fp,
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=False,
                               save_weights_only=False,
                               mode='auto')]
H = model.fit_generator(
  train_gen,
  steps_per_epoch=300,
  validation_data=val_gen,
  validation_steps=100,
  epochs=EPOCHS,
  callbacks=callbacks)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
300/300 [==============================] - 157s 523ms/step - loss: 0.3911 - acc: 0.8377 - val_loss: 0.2608 - val_acc: 0.9371

Epoch 00001: saving model to malaria-model-01-0.26.hdf5
Epoch 2/50
300/300 [==============================] - 144s 480ms/step - loss: 0.2108 - acc: 0.9433 - val_loss: 0.2132 - val_acc: 0.9480

Epoch 00002: saving model to malaria-model-02-0.21.hdf5
Epoch 3/50
300/300 [==============================] - 145s 483ms/step - loss: 0.1716 - acc: 0.9540 - val_loss: 0.3525 - val_acc: 0.8432

Epoch 00003: saving model to malaria-model-03-0.35.hdf5
Epoch 4/50
300/300 [==============================] - 147s 489ms/step - loss: 0.1554 - acc: 0.9559 - val_loss: 0.1947 - val_acc: 0.9540

Epoch 00004: saving model to malaria-model-04-0.1

In [0]:
!pip install kaggle 


In [0]:
!cat /var/log/colab-jupyter.log

{"name":"app","hostname":"d9227c965a1a","pid":37,"type":"jupyter","level":30,"msg":"Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret","time":"2019-02-09T14:05:11.280Z","v":0}
{"name":"app","hostname":"d9227c965a1a","pid":37,"type":"jupyter","level":40,"msg":"All authentication is disabled.  Anyone who can connect to this server will be able to run code.","time":"2019-02-09T14:05:11.338Z","v":0}
{"name":"app","hostname":"d9227c965a1a","pid":37,"type":"jupyter","level":30,"msg":"google.colab serverextension initialized.","time":"2019-02-09T14:05:11.400Z","v":0}
{"name":"app","hostname":"d9227c965a1a","pid":37,"type":"jupyter","level":30,"msg":"Serving notebooks from local directory: /","time":"2019-02-09T14:05:11.402Z","v":0}
{"name":"app","hostname":"d9227c965a1a","pid":37,"type":"jupyter","level":30,"msg":"0 active kernels","time":"2019-02-09T14:05:11.403Z","v":0}
{"name":"app","hostname":"d9227c965a1a","pid":37,"type":"jupyter","level"

In [0]:
from google.colab import files
files.download('model_cell.hdf5')

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.models import load_model
from skimage.transform import resize
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from sklearn.metrics import classification_report

classes = ['Parasitized', 'Uninfected']

def test_model(path, on_test_batch=False, image_path=None):
    rn50 = load_model("malaria-model-40-0.11.hdf5")
    print ('[INFO]Model Loaded Successfully!')

    if on_test_batch:
        val_aug = ImageDataGenerator(rescale=1.0/255)

        # Initialize the test generator
        test_gen = val_aug.flow_from_directory(
                        TEST_PATH,
                        class_mode="categorical",
                        target_size=(64, 64),
                        color_mode="rgb",
                        shuffle=False,
                        batch_size=BS)

        print ('[INFO]Evaluating the Model...')

        num_test_imgs = len(os.listdir(TEST_PATH + '/Uninfected')) + \
                        len(os.listdir(TEST_PATH + '/Parasitized'))

        test_gen.reset()
        pred_idxs = rn50.predict_generator(test_gen,
            steps=(num_test_imgs // BS) + 1)

        print (pred_idxs.shape, num_test_imgs)

        pred_idxs = np.argmax(pred_idxs, axis=1)

        print (classification_report(test_gen.classes, pred_idxs,
            target_names=test_gen.class_indices.keys()))

    else:
        if image_path == None:
            raise Exception('Path to image is None')

        img = plt.imread("datasets/cimages_val/Uninfected/C241NThinF_IMG_20151207_124643_cell_136.png")
        img = resize(img, (64, 64, 3))
        img = img[np.newaxis, :]
        predict = rn50.predict(img)
        print(predict)
        print ('The Image passed is of class: {}'.format(classes[np.argmax(predict[0])]))


    print ('[INFO]Model Evaluated Successfully!!')
path = "F"
test_model(path, on_test_batch=False, image_path=True)

[INFO]Model Loaded Successfully!


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[[0.00182724 0.99817276]]
The Image passed is of class: Uninfected
[INFO]Model Evaluated Successfully!!


In [13]:
from google.colab import files
files.download('malaria-model-40-0.11.hdf5') 

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 38278, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil